In [1]:
import mlflow.sklearn
import pandas as pd

import my_functions as mf

In [2]:
# Import data for prediction

data = pd.read_csv(f'./data/amazon_22_quarters.csv',
                   parse_dates=['EffectiveDate'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554079 entries, 0 to 554078
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   SKU                  554079 non-null  object        
 1   OfferTermCode        554079 non-null  object        
 2   RateCode             554079 non-null  object        
 3   PricePerUnit         554079 non-null  float64       
 4   instanceType         554079 non-null  object        
 5   instanceFamily       554079 non-null  object        
 6   LeaseContractLength  554079 non-null  int64         
 7   PurchaseOption       554079 non-null  object        
 8   OfferingClass        554079 non-null  object        
 9   Product Family       554079 non-null  object        
 10  Location             554079 non-null  object        
 11  Current Generation   554079 non-null  object        
 12  vCPU                 554079 non-null  int64         
 13  Memory        

In [3]:
data = data[['PricePerUnit', 'LeaseContractLength', 'PurchaseOption', 'vCPU',
             'Memory', 'operatingSystem', 'networkPerformance', 'StorageSize',
             'DiskType', 'Quarter', 'instanceType']]

# Prepare the Data

In [4]:
# # %%  =========== Prepare the Data for regression============

# # Map binary categorical columns to numerical

# categorical_binary = ['PurchaseOption']
# data[categorical_binary] = data[categorical_binary].apply(mf.binary_map)

# # Write the categorical values as a list
# # categorical = ['operatingSystem', 'DiskType', 'Location', 'instanceFamily']
# categorical = ['operatingSystem', 'DiskType']
# categorical2numeric = pd.get_dummies(
#     data[categorical], drop_first=False)

# categorical2numeric_pred = pd.get_dummies(
#     data[categorical], drop_first=False)

# # Add the above results to the original dataframe df
# data = pd.concat([data, categorical2numeric], axis=1)
# data.drop(columns=categorical, axis=1, inplace=True)

In [5]:
# Take instanceType as input from the user

instance_type = "g5.xlarge"

# instance_type = input("Enter the instance type: ")

# filter by instance type data
filtered_data = data[data['instanceType'] == instance_type]

# create df for actual prices
actual_prices = filtered_data.PricePerUnit


# drop instance type as it is not useful for the model
filtered_data.drop(
    columns=['PricePerUnit', 'instanceType'], axis=1, inplace=True)

/tmp/ipykernel_11342/1849532876.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.drop(


In [6]:
# RUN_ID = input("Enter the model RUN ID you want to use: ")

logged_model = "file:///media/gfragi/data/BarraCuda/mlruns/mlruns/977346654452183974/069a6067efd8495d85bc3a76d2742640/artifacts/catboost_model"

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

: 

: 

In [ ]:
# Predict on a Pandas DataFrame.
predictions = loaded_model.predict(pd.DataFrame(filtered_data))

# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(predictions, columns=['Predictions'])

# Save the predictions to a CSV file
predictions_df.to_csv(
    f'predResults/{instance_type}_predictions.csv', index=False)

In [ ]:
# Plot prdicted vs actual
import plotly.graph_objects as go

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(actual_prices))),
              y=actual_prices, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=list(range(len(predictions))),
              y=predictions, mode='lines', name='Predicted'))

fig.update_layout(
    title=f'Actual vs. Predicted Prices for {instance_type}]',
    xaxis=dict(title='Samples'),
    yaxis=dict(title='Price')
)

fig.show()

# Save the Plotly figure as an HTML file
# html_path = "temp/actual_predicted.html"
# pio.write_html(fig, html_path)

# # Log the HTML file as an artifact in MLflow
# mlflow.log_artifact(html_path)